# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
!pip install hf_transfer
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Xwin-LM-70B-V0.1-GGUF \
    --local-dir . \
    --local-dir-use-symlinks False \
    --include='*Q4_K_M*gguf*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 13.8 MB/s eta 0:00:00
downloading https://huggingface.co/TheBloke/Xwin-LM-70B-V0.1-GGUF/resolve/0f9c5cbd9ccbca39c3ec8b5c11d03cab76e2bedb/xwin-lm-70b-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpd3xdetx_
xwin-lm-70b-v0.1.Q4_K_M.gguf: 100% 41.4G/41.4G [02:27<00:00, 281MB/s]
/content


## 🚬Smoke test

`llama.cpp` は、デフォルトではメモリマップされたファイルを使用する。`--no-mmap` を有効化する。

[llama.cpp](https://github.com/ggerganov/llama.cpp/issues/2589#issuecomment-1693312217) を参照。

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=7116587 sha256=294269dd1934a01fa3f240729eef000f96d80cb542e1cf7687a51e890bee0194
  Stored in directory: /root/.cache/pip/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="./xwin-lm-70b-v0.1.Q4_K_M.gguf",
    n_gpu_layers=75,
    use_mmap=False,
    seed=42,
    n_ctx=2048,
    n_threads=11,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# Simple inference example
# Question: https://www.reddit.com/r/LocalLLaMA/comments/13zz8y5/what_questions_do_you_ask_llms_to_check_their/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
prompt = "A chat between a curious user and an artificial intelligence assistant. " \
        "The assistant gives helpful, detailed, and polite answers to the user's questions. " \
        "USER: Is it morally okay to kill mosquitoes? " \
        "ASSISTANT:"

output = llm(
  prompt, # Prompt
  temperature=0.7,
  echo=False,
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  seed=42,
  max_tokens=-1,  # Generate up to 512 tokens
)
print(output["choices"][0]["text"])

Llama.generate: prefix-match hit


 The question of whether it is morally acceptable to kill mosquitoes sparks a debate with valid points on both sides. Here are some arguments for and against the moral acceptability of killing mosquitoes:

Arguments for killing mosquitoes:

1. Self-defense: Mosquitoes can transmit diseases like malaria, dengue, and Zika, which can cause severe illness or even death in humans. By killing mosquitoes, we are protecting ourselves from these potential threats.
2. Reducing suffering: Mosquito bites can be itchy and uncomfortable, and some people have severe allergic reactions to their bites. Eliminating mosquitoes can reduce the overall suffering caused by their presence.
3. Ecological balance: In some cases, mosquitoes may become overpopulated in certain areas, leading to an imbalance in the ecosystem. By controlling their population, we can help maintain a healthier environment.
4. Human welfare: Mosquitoes can negatively impact human welfare by disrupting outdoor activities, causing sleep

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import jsonlines

def generate_text_with_llama(input_text):
    # Create a chat completion using the Llama object
    prompt = f"好奇心旺盛なユーザとAIアシスタントが会話しています。" \
            f"アシスタントは、ユーザの質問に対して、親切・詳細で役立つ回答を提供します。 " \
            f"USER: {input_text} " \
            f"ASSISTANT:"
    output = llm(
        prompt, # Prompt
        temperature=0.7,
        echo=False,
        stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
        seed=42,
        max_tokens=-1,
    )
    return output["choices"][0]["text"]

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text_with_llama(prompt)

            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")

            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)


Llama.generate: prefix-match hit


Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A.  1. 目標設定と確認: 自分の仕事に関連した短期・中期・長期の目標を設定し、定期的に確認することで、自分の仕事がどのように進んでいるかを理解し、熱意を保つことができます。
2. 成功の記録: 仕事で成功したプロジェクトや業績を書き留めることで、自分の成果を誇りに思い、熱意が復活することがあります。また、過去の成功事例を再び実現する動機づけにもなります。
3. スキルアップ: 仕事に関連したスキルを磨くことで、仕事への自信が向上し、熱意が復活することがあります。また、新しいスキルを身につけることで、仕事の幅も広がり、さらなる成功を目指せるようになります。
4. チームワークやコミュニケーションの重視: 同僚やチームメイトと積極的にコミュニケーションを取ることで、仕事への熱意を共有し合い、一緒に向上を目指すことができます。
5. ワークライフバランスの重視: 仕事と生活のバランスを大切にし、仕事以外の時間でリフレッシュやストレス蓄積の解消を行うことで、仕事への熱意を再び取り戻すことができます。



Llama.generate: prefix-match hit


Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A.  「クマとアザラシの物語」

第1章：クマの旅立ち

ある日、クマは森の中で、海が見たいと思った。彼は、家から遠く離れた場所に住むアザラシに会いたいとも考えていた。それ故、クマは旅立ちの日を定めた。

第2章：海辺への道のり

クマは森を出て、海辺までの道のりについた。道中、彼は沢山の動物と出会い、彼らから様々な助言を受けた。そして、彼は美しい海辺に到着した。

第3章：アザラシとの出会い

クマが海辺で立ち寄った町で、彼はアザラシと出会った。アザラシは、町中の人々と友達になれるように協力してくれた。クマはアザラシに感謝し、彼らが仲良くなることを願った。

第4章：友情の証

クマとアザラシは、一緒に遊ぶようになりました。彼らは海で遊泳し、砂浜で日光浴を楽しんだ。そして、彼らはお互いの文化や生活を教え合った。

第5章：家への帰途

クマは、アザラシとの素晴らしい時間を送りながら、家への帰途を考えるようになった。彼は、この旅で得た経験や友情を大切にし、今後もアザラシとの関係を大事にしたいと思った。

第6章：故郷への帰還

クマは、海辺での素晴らしい記憶を持ち、故郷へと帰還した。彼は家に帰ってからも、アザラシとの友情を大切にし続けることを誓った。

エピローグ：遠い友情

クマは、家に帰ってからもアザラシと手紙を交わし、お互いのことを気にかけ合った。彼らの友情は、時間と距離を超えて続くこととなり、二匹の心に永遠に残ることとなった。



Llama.generate: prefix-match hit


...

Q. 漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。

1. 立派　A. すばらしい
2. 開拓　B. ある事柄について知っている内容。
3. 知識　C. 物事に味わいや価値を加える
4. 賞賛　D. 新しい領域を切り開く
5. 味付け　E. 高い評価を与える
A.  1. 立派 - E. 高い評価を与える
2. 開拓 - D. 新しい領域を切り開く
3. 知識 - B. ある事柄について知っている内容。
4. 賞賛 - C. 物事に味わいや価値を加える
5. 味付け - A. すばらしい



Llama.generate: prefix-match hit


Q. 次の物語の展開を予想してみましょう。

ある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。
A.  謎めいた人物と出会った主人公は、その人物から不思議な力を授けられます。その力は、主人公が困難な状況に陥ると、自動的に現れて主人公を救うものです。

主人公はこの不思議な力に感謝し、それを使ってさまざまな困難を乗り越えていきます。しかし、ある時、主人公は自分がこの力に頼りすぎていることに気付きます。そして、力を捨てて自分自身で戦うことを決意します。

主人公は、自分の意志で得た力を使って、最終的には困難を乗り越え、幸せな生活を取り戻します。そして、この物語は、自分自身で立ち向かうことの大切さを伝える教訓的な結末を迎えます。



Llama.generate: prefix-match hit


Q. 次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。

1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 昔、あるところに老人と孫がいました。
5. 老人は微笑みながら次のように答えました。

A.  1. 昔、あるところに老人と孫がいました。
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 老人は微笑みながら次のように答えました。
5. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」

